<a href="https://colab.research.google.com/github/sumu97/Study/blob/main/NLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 57.4 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-03-27 23:17:09--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sys
import requests
import json
from tqdm import tqdm
from urllib.parse import urlparse, parse_qs

In [3]:
df = pd.read_csv('crawl_100.csv')

In [4]:
df

,Unnamed: 0,title,price
0,0,"평점5.0""연세대브랜드전문가"" 감성,모던 로고제작해 드립니다.","65,000원~"
1,1,로고 디자인 공모전 488회우승 명함이벤트 로고 제작해 드립니다.,"80,000원~"
2,2,"""2년 연속 크몽 어워즈 대상"" 1위의 실력을 보여 드립니다.","100,000원~"
3,3,"""9년연속 만족1위 + 로고 랭키1위"" 디자인파워 보여 드립니다.","100,000원~"
4,4,로고 CI BI 최신트랜드를 반영한 로고디자인 제작해 드립니다.,"100,000원~"
...,...,...,...
1795,1795,저화질 이미지를 ai 파일로 작업해 드립니다.,"5,000원~"
1796,1796,내 사업처럼 책임감있게 디자인 해 드립니다.,"200,000원~"
1797,1797,실제 사용한 전문적이고 깔끔한 PPT피피티 템플릿 드립니다.,"12,000원"
1798,1798,"소개서,제안서용 완성도 높은 PPT 피피티 템플릿 3개 드립니다.","15,000원"


가격 평균내고 기준 이상이면 1, 미만이면 0으로 분류

In [7]:
df['price'] = df['price'].str.replace("[,~원]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [13]:
df['title'] = df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [14]:
df

,Unnamed: 0,title,price
0,0,평점연세대브랜드전문가 감성모던 로고제작해 드립니다,65000
1,1,로고 디자인 공모전 회우승 명함이벤트 로고 제작해 드립니다,80000
2,2,년 연속 크몽 어워즈 대상 위의 실력을 보여 드립니다,100000
3,3,년연속 만족위 로고 랭키위 디자인파워 보여 드립니다,100000
4,4,로고 최신트랜드를 반영한 로고디자인 제작해 드립니다,100000
...,...,...,...
1795,1795,저화질 이미지를 파일로 작업해 드립니다,5000
1796,1796,내 사업처럼 책임감있게 디자인 해 드립니다,200000
1797,1797,실제 사용한 전문적이고 깔끔한 피피티 템플릿 드립니다,12000
1798,1798,소개서제안서용 완성도 높은 피피티 템플릿 개 드립니다,15000
